# Capstone Project
### this notebook will be mainly used for the capstone project.

In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests

In [2]:
wiki_link="https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
page=requests.get(wiki_link).text
soup = BeautifulSoup(page,'xml')
#soup

# Extract data from table


In [3]:
tables = soup.find('table')

Postcode      = []
Borough       = []
Neighbourhood = []



## Reading table data

In [4]:
for table in tables.find_all('tr'):
    count=1
    Borough_var=-1
    postcode_var=-1
    Neighbor_var=-1
    t= table.find_all('td')
    for i in t:
        if count==1:
            postcode_var=i.text
        if count==2:
            Borough_var=i.text
            tag_borough=i.find('a')
        if count==3:
            Neighbor_var=str(i.text).strip()
            tag_neighbor=i.find('a')
        
        count+=1
        #print(t)
    if (Borough_var == 'Not assigned'):
        continue
    if (Neighbor_var=='Not assigned') and (Borough_var != 'Not assigned'):
        Neighbor_var=Borough_var
    try:
        if (tag_borough is None):
            continue
    except:
        pass

    
    Postcode.append(postcode_var)
    Borough.append(Borough_var)
    Neighbourhood.append(Neighbor_var)
                         
        

## Converting to table dataframe

In [5]:
l=len(Postcode)
m=set(Postcode)
print(len(m))

Post_u      = []
Bor_u       = []
Neighbour_u = []


for post in m:
    p_var = ''; b_var = ''; n_var = ''; 
    for post_id, n in enumerate(Postcode):
        if post == n:
            p_var = post;
            b_var = Borough[post_id]
            if n_var == '': 
                n_var = Neighbourhood[post_id]
            else:
                n_var = n_var + ', ' + Neighbourhood[post_id]
    Post_u.append(p_var)
    Bor_u.append(b_var)
    Neighbour_u.append(n_var)

101


In [6]:
toronto={'Postcode':Post_u,'Borough': Bor_u,'Neighbourhood':Neighbour_u}
df=pd.DataFrame.from_dict(toronto)

# Add Longitude and lattitude

In [7]:
import io
url="http://cocl.us/Geospatial_data"
s=requests.get(url).content
c=pd.read_csv(io.StringIO(s.decode('utf-8')))
c.columns = ['Postcode', 'Latitude', 'Longitude']
df = pd.merge(c, df, on='Postcode')
df = df[['Postcode', 'Borough', 'Neighbourhood', 'Latitude', 'Longitude']]


In [8]:
df.head(10)

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848
